## Library

In [1]:
import pandas as pd
import numpy as np
import re

## Getting Data

In [2]:
import os 

path = os.path.join(os.getcwd(),'Data/training_data.csv')
col_names = ['target','ids','date','flag','user','text']

df = pd.read_csv(path)

In [3]:
df.columns = ['target','ids','date','flag','user','text']

In [4]:
# Data for Detection
data = df[['target','text']]

In [24]:
def clean_text(text):
    text = text.lower().strip()
    text = re.sub(r"http\S+|www\S+", "", text)  # Xóa URL
    text = re.sub(r'[^a-zA-Z\s]', " ", text)   # Chỉ giữ chữ cái và khoảng trắng
    text = " ".join(text.split())  # Chuẩn hóa khoảng trắng
    return text

data['text'] = data['text'].astype(str).apply(clean_text)

/tmp/ipykernel_27412/40761610.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].astype(str).apply(clean_text)


## Detection

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score 


In [26]:
x = data['text'].values
y = data['target'].values

In [27]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 19)
vectorizer = TfidfVectorizer(stop_words = 'english')
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [16]:
from sklearn.linear_model import LogisticRegression      # Logistic Regression
from sklearn.naive_bayes import MultinomialNB # Naive Bayes

In [ ]:
models = {
    'lr_model' : LogisticRegression(max_iter = 1000),
    'nv_model' : MultinomialNB(),
}

def train_and_evaluate_models(models, x_train, y_train, x_test, y_test):
    results = {}  # Lưu kết quả đánh giá
    
    for name, model in models.items():
        print(f"\nTraining model: {name} ...")
        
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        acc = accuracy_score(y_test, y_pred)
        
        print(f"Accuracy for {name}: {acc:.4f}")
        results[name] = {"accuracy": acc}
    
    return results

In [29]:
results = train_and_evaluate_models(models, x_train, y_train, x_test, y_test)


Training model: lr_model ...
Accuracy for lr_model: 0.7808

Training model: nv_model ...
Accuracy for nv_model: 0.7612


In [36]:
def Sentiment(text):
    text = vectorizer.transform([text])
    model = LogisticRegression(max_iter= 1000)
    model.fit(x_train,y_train)
    pre = model.predict(text)

    return "Possitive" if pre[0] == 0 else "Negative"

In [38]:
text = input("Nhap van ban: ")
print(Sentiment(text))

Possitive
